# Demo 1

Preview of output as more DCT channels arrive

In [1]:
import numpy as np
from PIL import Image
from scipy.fft import dctn, idctn
from matplotlib import pylab as pylab
from matplotlib import pyplot as plt

%matplotlib inline
pylab.rcParams['figure.figsize'] = (8, 8)


In [2]:
def mdct4(x):
    N = x.shape[0]
    if N%4 != 0:
        raise ValueError("MDCT4 only defined for vectors of length multiple of four.")
    M = N // 2
    N4 = N // 4
    
    rot = np.roll(x, N4)
    rot[:N4] = -rot[:N4]
    t = np.arange(0, N4)
    w = np.exp(-1j*2*np.pi*(t + 1./8.) / N)
    c = np.take(rot,2*t) - np.take(rot, N-2*t-1) \
        - 1j * (np.take(rot, M+2*t) - np.take(rot,M-2*t-1))
    c = (2./np.sqrt(N)) * w * np.fft.fft(0.5 * c * w, N4)
    y = np.zeros(M)
    y[2*t] = np.real(c[t])
    y[M-2*t-1] = -np.imag(c[t])
    return y


def imdct4(x):
    N = x.shape[0]
    if N%2 != 0:
        raise ValueError("iMDCT4 only defined for even-length vectors.")
    M = N // 2
    N2 = N*2
    
    t = np.arange(0,M)
    w = np.exp(-1j*2*np.pi*(t + 1./8.) / N2)
    c = np.take(x,2*t) + 1j * np.take(x,N-2*t-1)
    c = 0.5 * w * c
    c = np.fft.fft(c,M)
    c = ((8 / np.sqrt(N2))*w)*c
    
    rot = np.zeros(N2)
    
    rot[2*t] = np.real(c[t])
    rot[N+2*t] = np.imag(c[t])
    
    t = np.arange(1,N2,2)
    rot[t] = -rot[N2-t-1]
    
    t = np.arange(0,3*M)
    y = np.zeros(N2)
    y[t] = rot[t+M]
    t = np.arange(3*M,N2)
    y[t] = -rot[t-3*M]
    return y




In [3]:
X = np.random.rand(1000)
Y1 = mdct4(X[:100])
Y2 = mdct4(X[50:150])
Z1 = imdct4(Y1)
Z2 = imdct4(Y2)
print(np.linalg.norm(Z1[50:100] + Z2[:50] - 2*X[50:100])) # should be about 0
print(Z1[50:100] + Z2[:50] - 2*X[50:100]) # should be about 0

2.520719752872653e-15
[-2.22044605e-16  4.44089210e-16  4.44089210e-16  6.66133815e-16
  2.22044605e-16  8.88178420e-16  4.44089210e-16  2.22044605e-16
  4.44089210e-16  0.00000000e+00  0.00000000e+00  3.33066907e-16
  1.66533454e-16  4.44089210e-16  4.44089210e-16  0.00000000e+00
  4.44089210e-16  2.22044605e-16 -2.22044605e-16  4.44089210e-16
  3.88578059e-16  0.00000000e+00 -2.22044605e-16  0.00000000e+00
  4.44089210e-16  2.22044605e-16  4.44089210e-16  3.88578059e-16
  1.66533454e-16  2.22044605e-16  0.00000000e+00  2.22044605e-16
  2.22044605e-16  7.21644966e-16  1.11022302e-16 -2.22044605e-16
  2.22044605e-16 -5.55111512e-17  0.00000000e+00  0.00000000e+00
 -2.22044605e-16  0.00000000e+00  3.33066907e-16  6.66133815e-16
 -2.22044605e-16  4.44089210e-16  6.66133815e-16 -2.22044605e-16
  1.11022302e-16 -4.44089210e-16]


In [4]:
from scipy.signal.windows import kaiser_bessel_derived

Y1 = mdct4(X[:100] * kaiser_bessel_derived(100, beta=16))
Y2 = mdct4(X[50:150] * kaiser_bessel_derived(100, beta=16))
Z1 = imdct4(Y1) * kaiser_bessel_derived(100, beta=16)
Z2 = imdct4(Y2) * kaiser_bessel_derived(100, beta=16)
print(np.linalg.norm(Z1[50:100] + Z2[:50] - 2*X[50:100])) # should be about 0
print(Z1[50:100] + Z2[:50] - 2*X[50:100]) # should be about 0

3.8148663449272027
[-0.62070193 -0.30649835 -0.0076492  -0.63116941 -0.73188012 -0.4624331
 -0.18591169 -0.98733321 -0.20561803 -0.64500553 -0.82530009 -0.29526532
 -0.24841919 -0.69975991 -0.01489196 -0.74688528 -0.56974057 -0.96070493
 -0.47298384 -0.60985547 -0.12053706 -0.30661678 -0.46662361 -0.92511008
 -0.84384036 -0.01651942 -0.67500198 -0.09992968 -0.18602465 -0.42748648
 -0.05436838 -0.19243772 -0.10106833 -0.20792075 -0.02022561 -0.26422467
 -0.94196433 -0.04391223 -0.66379064 -0.68035738 -0.10292523 -0.27599249
 -0.09992275 -0.90203178 -0.27688058 -0.65530785 -0.40573073 -0.81052956
 -0.00552732 -0.98261133]
